<a href="https://colab.research.google.com/github/UdaraChamidu/Full-Stack-Data-Science-Project/blob/main/extract_video_names.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For this entire project i have used only titles of a you tube videos in a channel. because the you tybe transcript api did not worked.

In [63]:
pip install youtube-transcript-api==0.6.2

In [64]:
import requests # api calls
import json # CONVERT TO JSON
import polars as pl # faster version of pandas
from google.colab import userdata # load secreats

from youtube_transcript_api import YouTubeTranscriptApi

In [65]:
import os
my_key = userdata.get('my_key')

In [66]:
def getVideoRecords(response: requests.models.Response) -> list:
    """
        Function to extract YouTube video data from GET request response
    """

    video_record_list = []

    for raw_item in json.loads(response.text)['items']:

        # only execute for youtube videos
        if raw_item['id']['kind'] != "youtube#video":
            continue

        video_record = {}
        video_record['video_id'] = raw_item['id']['videoId']
        video_record['datetime'] = raw_item['snippet']['publishedAt']
        video_record['title'] = raw_item['snippet']['title']
        #video_record['transcript'] = raw_item['snippet']['transcript']
        #video_record['description'] = raw_item['snippet']['description']

        video_record_list.append(video_record)

    return video_record_list

In [67]:
# define channel ID
channel_id = 'UCa9gErQ9AE5jT2DZLjXBIdA'
# https://www.youtube.com/@SuprOrdinary  recomended by chat GPT
# define url for API

url = 'https://www.googleapis.com/youtube/v3/search'

# initialize page token
page_token = None

# intialize list to store video data
video_record_list = []

In [68]:
%%time
# extract video data across multiple search result pages
while page_token != 0:
    # define parameters for API call
    # max result is reduced to 2. because it exceed the daily limit.
    params = {"key": my_key, 'channelId': channel_id, 'part': ["snippet","id"], 'order': "date", 'maxResults':2, 'pageToken': page_token}
    # make get request
    response = requests.get(url, params=params)

    # Check if the request was successful and the response contains 'items'
    if response.status_code == 200 and 'items' in json.loads(response.text):
        # append video records to list
        video_record_list += getVideoRecords(response)

        try:
            # grab next page token
            page_token = json.loads(response.text)['nextPageToken']
        except KeyError:
            # if no next page token kill while loop
            page_token = 0
    else:
        # If the request was not successful or 'items' is missing, stop the loop
        print(f"API request failed with status code {response.status_code} or no items found.")
        print(response.text) # Print the response text for debugging
        page_token = 0

CPU times: user 393 ms, sys: 34.1 ms, total: 427 ms
Wall time: 34.5 s


In [69]:
# write data to file
os.makedirs('data')
pl.DataFrame(video_record_list).write_parquet('data/video-ids.parquet')
pl.DataFrame(video_record_list).write_csv('data/video-ids.csv')

In [70]:
video_record_list

[{'video_id': 'QxLXhE1fxc4',
  'datetime': '2025-08-31T08:00:23Z',
  'title': 'uv: The Fastest Way to Install (and Use) Python'},
 {'video_id': 'enBm0jLXLZ4',
  'datetime': '2025-08-24T08:00:44Z',
  'title': 'GitHub for AI Engineers (beginner-friendly guide)'},
 {'video_id': 'zKHSpwayPBU',
  'datetime': '2025-08-17T08:00:59Z',
  'title': 'Context Engineering Explained (5 Practical Tips)'},
 {'video_id': 'hugQUr4VwRA',
  'datetime': '2025-08-10T08:01:00Z',
  'title': 'How to Solve Problems with AI (not just use it)'},
 {'video_id': 'fAFJYbtTsC0',
  'datetime': '2025-07-27T08:00:45Z',
  'title': 'Fine-tuning LLMs for Tool Use (w/ Example Code)'},
 {'video_id': 'uZvR1tG4Dng',
  'datetime': '2025-07-20T15:01:27Z',
  'title': 'How I Found My Ideal Customer with AI (ICP Analysis)'},
 {'video_id': 'bNmC_N4X1lk',
  'datetime': '2025-07-13T15:01:02Z',
  'title': 'How to Solve Problems with Text Embeddings and ML (Dallas AI - Summer School 2025)'},
 {'video_id': 'U9XDqQmV1Hc',
  'datetime': '202

# Get the Titles

In [71]:
import polars as pl

In [72]:
def extract_text(transcript: list) -> str:
    """
        Function to extract text from transcript dictionary
    """

    text_list = [transcript[i]['text'] for i in range(len(transcript))]
    return ' '.join(text_list)

In [73]:
df = pl.read_parquet('data/video-ids.parquet')
print(df.head())

shape: (5, 3)
┌─────────────┬──────────────────────┬─────────────────────────────────┐
│ video_id    ┆ datetime             ┆ title                           │
│ ---         ┆ ---                  ┆ ---                             │
│ str         ┆ str                  ┆ str                             │
╞═════════════╪══════════════════════╪═════════════════════════════════╡
│ QxLXhE1fxc4 ┆ 2025-08-31T08:00:23Z ┆ uv: The Fastest Way to Install… │
│ enBm0jLXLZ4 ┆ 2025-08-24T08:00:44Z ┆ GitHub for AI Engineers (begin… │
│ zKHSpwayPBU ┆ 2025-08-17T08:00:59Z ┆ Context Engineering Explained … │
│ hugQUr4VwRA ┆ 2025-08-10T08:01:00Z ┆ How to Solve Problems with AI … │
│ fAFJYbtTsC0 ┆ 2025-07-27T08:00:45Z ┆ Fine-tuning LLMs for Tool Use … │
└─────────────┴──────────────────────┴─────────────────────────────────┘


In [74]:
import pandas as pd
pd.read_csv("data/video-ids.csv")

,video_id,datetime,title
0,QxLXhE1fxc4,2025-08-31T08:00:23Z,uv: The Fastest Way to Install (and Use) Python
1,enBm0jLXLZ4,2025-08-24T08:00:44Z,GitHub for AI Engineers (beginner-friendly guide)
2,zKHSpwayPBU,2025-08-17T08:00:59Z,Context Engineering Explained (5 Practical Tips)
3,hugQUr4VwRA,2025-08-10T08:01:00Z,How to Solve Problems with AI (not just use it)
4,fAFJYbtTsC0,2025-07-27T08:00:45Z,Fine-tuning LLMs for Tool Use (w/ Example Code)
...,...,...,...
146,MX7ymkYGiZ0,2020-12-21T00:24:45Z,The Wavelet Transform | Introduction &amp; Exa...
147,rPUytg38b6Q,2020-12-04T01:10:36Z,The Fast Fourier Transform | How does it (actu...
148,mj86XmfOniY,2020-11-15T20:41:53Z,"Time Series, Signals, &amp; the Fourier Transf..."
149,Gwz4zXPeP_Q,2020-11-12T22:58:00Z,biometricDahboard3 DEMO


# Data Processing

In [76]:
import polars as pl  # like pandas

from sentence_transformers import SentenceTransformer, util   # for text embedding

from sklearn.metrics import DistanceMetric  # evaluation
import numpy as np

import matplotlib.pyplot as plt

In [79]:
df = pl.read_parquet('data/video-ids.parquet')
df_eval = pl.read_csv('data/video-ids.csv')
df.head()

video_id,datetime,title
str,str,str
"""QxLXhE1fxc4""","""2025-08-31T08:00:23Z""","""uv: The Fastest Way to Install…"
"""enBm0jLXLZ4""","""2025-08-24T08:00:44Z""","""GitHub for AI Engineers (begin…"
"""zKHSpwayPBU""","""2025-08-17T08:00:59Z""","""Context Engineering Explained …"
"""hugQUr4VwRA""","""2025-08-10T08:01:00Z""","""How to Solve Problems with AI …"
"""fAFJYbtTsC0""","""2025-07-27T08:00:45Z""","""Fine-tuning LLMs for Tool Use …"


In [81]:
df_eval.head()

video_id,datetime,title
str,str,str
"""QxLXhE1fxc4""","""2025-08-31T08:00:23Z""","""uv: The Fastest Way to Install…"
"""enBm0jLXLZ4""","""2025-08-24T08:00:44Z""","""GitHub for AI Engineers (begin…"
"""zKHSpwayPBU""","""2025-08-17T08:00:59Z""","""Context Engineering Explained …"
"""hugQUr4VwRA""","""2025-08-10T08:01:00Z""","""How to Solve Problems with AI …"
"""fAFJYbtTsC0""","""2025-07-27T08:00:45Z""","""Fine-tuning LLMs for Tool Use …"


Embedding

In [82]:
# define "parameters"
# titeles loops through 3 embeddings...
# loop through every embeddings and generate embeddings.(for one title, 3 will find after that)
column_to_embed_list = ['title']
model_name_list = ["all-MiniLM-L6-v2", "multi-qa-distilbert-cos-v1", "multi-qa-mpnet-base-dot-v1"]

In [83]:
# generate embeddings for each combination of column and model

# initialize dict to keep track of all text embeddings
text_embedding_dict = {}

for model_name in model_name_list:

    #define embedding model
    model = SentenceTransformer(model_name)

    for column_name in column_to_embed_list:

        # define text embedding identifier
        key_name = model_name + "_" + column_name
        print(key_name)

        # generate embeddings for text under column_name
        %time embedding_arr = model.encode(df[column_name].to_list())
        print('')

        # append embeddings to dict
        text_embedding_dict[key_name] = embedding_arr

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

multi-qa-mpnet-base-dot-v1_title
CPU times: user 297 ms, sys: 11.7 ms, total: 309 ms
Wall time: 393 ms



In [84]:
embedding_arr

array([[-0.0579739 , -0.24201667, -0.2758952 , ..., -0.14767565,
        -0.10521438,  0.06014388],
       [-0.10298827, -0.04777742, -0.39999035, ..., -0.11524759,
        -0.00191435, -0.34256133],
       [-0.04870157, -0.5153795 , -0.27510655, ..., -0.00417584,
         0.07389141, -0.28586006],
       ...,
       [-0.16630918, -0.17924808, -0.36003548, ..., -0.00500476,
         0.1269911 ,  0.12001868],
       [ 0.10038812, -0.5096164 , -0.18251847, ..., -0.14485003,
        -0.22460009, -0.00664386],
       [ 0.10863414, -0.4984184 , -0.24904884, ..., -0.12101515,
        -0.16033797, -0.12821192]], dtype=float32)

In [85]:
embedding_arr.shape

(151, 768)

In [86]:
# all generated titles using the embedding models
text_embedding_dict

{'all-MiniLM-L6-v2_title': array([[-0.05386354,  0.02306378,  0.01648222, ...,  0.02966654,
          0.09049685, -0.01569938],
        [-0.06883831, -0.10490226, -0.02755976, ...,  0.10181764,
         -0.00645436, -0.00632363],
        [-0.02870849,  0.05076479, -0.00959319, ...,  0.15052605,
         -0.03831506,  0.043586  ],
        ...,
        [-0.14074452,  0.02683029,  0.00540426, ...,  0.06849577,
         -0.01954203,  0.03768773],
        [-0.08560575,  0.00512425, -0.08294972, ...,  0.06376527,
         -0.05695948, -0.04079673],
        [-0.04230064,  0.00334014, -0.04482029, ...,  0.07943333,
         -0.04418199, -0.04676233]], dtype=float32),
 'multi-qa-distilbert-cos-v1_title': array([[-0.00188188, -0.00793579,  0.02383152, ...,  0.04213658,
         -0.02075406, -0.00827226],
        [ 0.03711838,  0.01423494, -0.00633548, ...,  0.01067772,
         -0.01075286,  0.05702541],
        [ 0.018963  ,  0.0752207 , -0.03809123, ..., -0.01488493,
         -0.04399891,  0.0

In [ ]:
query_embedding_dict = {}

for model_name in model_name_list:

    #define embedding model
    model = SentenceTransformer(model_name)
    print(model_name)

    # embed query text
    %time embedding_arr = model.encode(df_eval['query'].to_list())
    print('')

    # append embedding to dict
    query_embedding_dict[model_name] = embedding_arr

Evaluate